# Liberias 

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

In [ ]:
#ruta para los csv en data/processed
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Subir un nivel
DATA_DIR = os.path.join(BASE_DIR, "data", "processed") 

# Rand-Forest

## Feaure Importance
Medimos la importancia de cada variable para ver las que vamosa considerar en nuestro modelo


In [ ]:
def evaluar_importancia_variables(data_dir):
    resultados = {}

    # Lista de archivos esperados
    file_paths = {
        "v1": os.path.join(data_dir, "Propensity_clean_v1.csv"),
        "v2": os.path.join(data_dir, "Propensity_clean_v2.csv"),
        "v3": os.path.join(data_dir, "Propensity_clean_v3.csv"),
        "v4": os.path.join(data_dir, "Propensity_clean_v4.csv"),
    }

    for version, path in file_paths.items():
        print(f"\n🔍 Evaluando importancia de variables en: {version}")

        # Cargar los datos
        df = pd.read_csv(path)

        # Verificar si la columna "Mas_1_coche" está presente y eliminar columnas problemáticas
        if "Mas_1_coche" in df.columns:
            y = df["Mas_1_coche"]
            X = df.drop(columns=["Mas_1_coche"], errors="ignore")
        # Eliminar columnas no numéricas si existen
        X = X.select_dtypes(include=['number'])

        # Dividir en entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Entrenar un Random Forest básico
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)

        # Obtener importancia de variables
        importances = rf.feature_importances_
        feature_importance_df = pd.DataFrame({"Variable": X.columns, "Importancia": importances})
        feature_importance_df = feature_importance_df.sort_values(by="Importancia", ascending=False)

        # Guardar resultados
        resultados[version] = feature_importance_df

        # Mostrar la tabla de importancia de variables
        display(feature_importance_df)

        # Graficar la importancia de las variables
        plt.figure(figsize=(10, 6))
        sns.barplot(x="Importancia", y="Variable", data=feature_importance_df)
        plt.title(f"Importancia de Variables en Random Forest - {version}")
        plt.show()

    return resultados


# Ejecutar la función con la ruta corregida
importancia_variables_resultados = evaluar_importancia_variables(DATA_DIR)


Como en cada csv hemos hecho un tratamiento distinto de los nulos , comparamos la importancia de cada variable en cada uno de ellos

In [ ]:
comparacion_df = pd.DataFrame()

for version, df_importance in importancia_variables_resultados.items():
    df_importance = df_importance.rename(columns={"Importancia": f"Importancia_{version}"})
    if comparacion_df.empty:
        comparacion_df = df_importance
    else:
        comparacion_df = comparacion_df.merge(df_importance, on="Variable", how="outer")


comparacion_df["Importancia_Promedio"] = comparacion_df.drop(columns=["Variable"]).mean(axis=1)
comparacion_df = comparacion_df.sort_values(by="Importancia_Promedio", ascending=False)
comparacion_df = comparacion_df.drop(columns=["Importancia_Promedio"])

display(comparacion_df)


,Variable,Importancia_v1,Importancia_v2,Importancia_v3,Importancia_v4
19,Tiempo,0.793701,0.806694,0.846439,0.838849
20,Zona_Renta,0.092084,0.082641,0.036537,0.040258
2,COSTE_VENTA,0.032018,0.033894,0.038409,0.037599
21,km_anno,0.019135,0.023683,0.025661,0.025243
11,PRODUCTO,0.012393,0.010270,0.009885,0.010297
3,Campanna1,0.005547,0.005535,0.005545,0.005543
8,FORMA_PAGO,0.004881,0.004794,0.005931,0.006506
17,TIPO_CARROCERIA,0.007483,0.005610,0.004194,0.004323
12,PROVINCIA,0.006006,0.004976,0.004784,0.005488
16,Revisiones,0.004407,0.004143,0.005255,0.005976


**Exclusión de la Variable `Tiempo`**

Tiempo porque es un parámetro que solo tienen los clientes con una segunda compra. Esto sesga la predicción, ya que no es representativo de todos los clientes, especialmente aquellos que no han realizado una segunda compra. 

In [28]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats import randint

# 📌 1️⃣ Definir la ruta base y la carpeta de datos procesados
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Subir un nivel desde "models"
DATA_DIR = os.path.join(BASE_DIR, "data", "processed")  # Ruta completa a "data/processed"

# 📌 2️⃣ Cargar el dataset (Asegúrate de que el archivo existe antes de cargarlo)
file_name = "Propensity_clean_v1.csv"  # Puedes cambiar a "Propensity_clean_v2.csv", "v3", "v4"
file_path = os.path.join(DATA_DIR, file_name)

if not os.path.exists(file_path):
    raise FileNotFoundError(f"⚠️ No se encontró el archivo: {file_path}")

df = pd.read_csv(file_path)

# 📌 3️⃣ Eliminar "Mas_1_coche" y "Tiempo"
y = df["Mas_1_coche"]
X = df.drop(columns=["Mas_1_coche", "Tiempo"], errors="ignore")

# 📌 4️⃣ Dividir en entrenamiento y prueba
X_train_all, X_test_all, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 5️⃣ Seleccionar las N variables más importantes
N = 9  # Número de variables más importantes
feature_importance = [
    "Zona_Renta", "COSTE_VENTA", "km_anno", "PRODUCTO",
    "TIPO_CARROCERIA", "Campanna1", "PROVINCIA", "FORMA_PAGO", "Revisiones"
]  # Cambia esta lista si es necesario

# Crear dataset con solo las variables más importantes
X_train_top = X_train_all[feature_importance]
X_test_top = X_test_all[feature_importance]

# 📌 6️⃣ Definir los hiperparámetros para `Random Search`
param_dist = {
    "n_estimators": randint(50, 300),
    "max_depth": [5, 10, 15, 20, None],
    "min_samples_split": randint(2, 10),
    "min_samples_leaf": randint(1, 5),
    "max_features": ["sqrt", "log2"],
    "bootstrap": [True, False]
}

# 📌 7️⃣ Entrenar `Random Search` con todas las variables
random_search_all = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)
random_search_all.fit(X_train_all, y_train)

# 📌 8️⃣ Entrenar `Random Search` con solo las variables más importantes
random_search_top = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)
random_search_top.fit(X_train_top, y_train)

# 📌 9️⃣ Mostrar los mejores hiperparámetros para cada caso
print("📌 Mejores Hiperparámetros con Todas las Variables:")
print(random_search_all.best_params_)

print("\n📌 Mejores Hiperparámetros con Solo las Variables Más Importantes:")
print(random_search_top.best_params_)


📌 Mejores Hiperparámetros con Todas las Variables:
{'bootstrap': False, 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 7, 'n_estimators': 103}

📌 Mejores Hiperparámetros con Solo las Variables Más Importantes:
{'bootstrap': False, 'max_depth': 15, 'max_features': 'log2', 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 113}


In [33]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats import randint

# 📌 1️⃣ Definir la ruta base y la carpeta de datos procesados
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(BASE_DIR, "data", "processed")

# 📌 2️⃣ Cargar el dataset
file_name = "Propensity_clean_v1.csv"  # Cambia entre "v1", "v2", "v3", "v4"
file_path = os.path.join(DATA_DIR, file_name)

if not os.path.exists(file_path):
    raise FileNotFoundError(f"⚠️ No se encontró el archivo: {file_path}")

df = pd.read_csv(file_path)

# 📌 3️⃣ Eliminar "Mas_1_coche" y "Tiempo"
y = df["Mas_1_coche"]
X = df.drop(columns=["Mas_1_coche", "Tiempo"], errors="ignore")

# 📌 4️⃣ Dividir en entrenamiento y prueba
X_train_all, X_test_all, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 5️⃣ Seleccionar las N variables más importantes
N = 9
feature_importance = [
    "Zona_Renta", "COSTE_VENTA", "km_anno", "PRODUCTO",
    "TIPO_CARROCERIA", "Campanna1", "PROVINCIA", "FORMA_PAGO", "Revisiones"
]

X_train_top = X_train_all[feature_importance]
X_test_top = X_test_all[feature_importance]

# 📌 6️⃣ Definir los hiperparámetros EXPANDIDOS para `Random Search`
param_dist = {
    "n_estimators": [ 200, 300, 500],  # Aumentamos el número de árboles
    "max_depth": [ None],  # Probamos árboles más grandes
    "min_samples_split": [ 15, 20],  # Más opciones para dividir nodos
    "min_samples_leaf": [ 5, 10],  # Probamos hojas más grandes
    "max_features": [ None],  # Incluimos `None` (usar todas las features)
    "bootstrap": [True, False],  # Seguimos probando ambos
    "class_weight": [None],  # Para manejar desequilibrio de clases
    "max_leaf_nodes": [None, 50, 100],  # Limitamos el número de nodos hoja
    "warm_start": [False]  # No usamos `True` por ahora, pero podríamos probarlo
}

# 📌 7️⃣ Entrenar `Random Search` con todas las variables
random_search_all = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=50,  # Aumentamos a 50 iteraciones para mejor exploración
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)
random_search_all.fit(X_train_all, y_train)

# 📌 8️⃣ Entrenar `Random Search` con solo las variables más importantes
random_search_top = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)
random_search_top.fit(X_train_top, y_train)

# 📌 9️⃣ Mostrar los mejores hiperparámetros para cada caso
print("📌 Mejores Hiperparámetros con Todas las Variables:")
print(random_search_all.best_params_)

print("\n📌 Mejores Hiperparámetros con Solo las Variables Más Importantes:")
print(random_search_top.best_params_)


📌 Mejores Hiperparámetros con Todas las Variables:
{'warm_start': False, 'n_estimators': 200, 'min_samples_split': 15, 'min_samples_leaf': 5, 'max_leaf_nodes': None, 'max_features': None, 'max_depth': None, 'class_weight': None, 'bootstrap': True}

📌 Mejores Hiperparámetros con Solo las Variables Más Importantes:
{'warm_start': False, 'n_estimators': 200, 'min_samples_split': 20, 'min_samples_leaf': 5, 'max_leaf_nodes': None, 'max_features': None, 'max_depth': None, 'class_weight': None, 'bootstrap': True}
